In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## _Sorting out cells that made it to landfall (used the globe.is_land() library)_

In [2]:
track = pd.read_csv('../saved-files/threshold-all/final-tracks-threshold-all.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,3,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,155,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,3,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,311,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,3,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,463,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,3,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,610,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,3,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,299,43,144.743717,239.312025,4,15,35461,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,12303,0 days 00:05:00,122.013418,15.240493,41.116443,37.803245
25142,300,49,146.461260,240.148842,5,15,35519,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,12303,0 days 00:10:00,120.179000,14.702386,40.106802,77.594511
25143,298,82,129.807126,264.363306,8,20,35414,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.189807e+06,181363.305965,70.221738,14.734522,12312,0 days 00:00:00,119.604279,15.301029,39.603226,71.626426
25144,299,75,132.210510,264.802250,9,20,35474,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.192211e+06,181802.250223,70.199616,14.740749,12312,0 days 00:05:00,117.272271,15.240493,41.116443,71.624960


In [3]:
track.head()   

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,3,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,155,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,3,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,311,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,3,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,463,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,3,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,610,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,3,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374


In [4]:
%%time
cell_landfall = []
for i, j in enumerate(track['cell']):
#     # # Check if a point is on land:
    lat = track['latitude'][i]
    lon = track['longitude'][i]
    is_on_land = globe.is_land(lat, lon)
    if is_on_land:
        # if ((lat >= 69.0) & (lon >= 12.0)):
            # print(f"lat={lat}, lon={lon} is on land: {is_on_land} -- cell_id={j}")
        cell_landfall.append(j)

CPU times: user 2.65 s, sys: 964 µs, total: 2.65 s
Wall time: 2.72 s


In [5]:
%%time
cells_landfall=np.unique(cell_landfall)
cells_landfall

CPU times: user 292 µs, sys: 25 µs, total: 317 µs
Wall time: 322 µs


array([   42,    79,   118, ..., 12277, 12294, 12295])

## _Algorithm to track cells back in time that made it to landfall_

In [6]:
%%time
combined_cells_landfalls = []
for i, cell_ids in enumerate(np.unique(cells_landfall)): # looped through tracks found with AMF1 site
    if cell_ids in track['cell'].values: # checking if those cells(tracks) are within the original tracked dataframe
        # print(f'Cell {cell_ids} present: made it to landfall 😀')
        track_c = track[track['cell'] == cell_ids] # display it if it is within the tracked dataframe
        combined_cells_landfalls.append(track_c) # append the results you get a new dataFrame calld combined_tracks_AMF1
    else:
        print('Cells not present 🥲')
        
# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
combined_cells_landfall_present = pd.concat(combined_cells_landfalls)
combined_cells_landfall_present = combined_cells_landfall_present.reset_index(drop=True)
combined_cells_landfall_present['landfall'] = globe.is_land(combined_cells_landfall_present['latitude'], combined_cells_landfall_present['longitude'])

CPU times: user 471 ms, sys: 11 ms, total: 482 ms
Wall time: 486 ms


In [7]:
combined_cells_landfall_present.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall
0,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,197675.949726,68.422543,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True
1,1,68,325.560542,282.342019,49,10,191,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.385561e+06,199342.018987,68.430539,14.776648,42,0 days 00:05:00,102.818375,12.974534,35.577388,58.710257,True
2,2,71,326.725123,282.815677,49,10,351,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.386725e+06,199815.677294,68.419587,14.785620,42,0 days 00:10:00,103.725137,12.780750,34.063904,58.711762,True
3,0,145,333.698552,288.034937,66,15,79,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.393699e+06,205034.937276,68.352148,14.895784,79,0 days 00:00:00,108.538869,13.245867,34.567482,187.081946,True
4,1,154,333.912558,289.040029,57,15,239,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.393913e+06,206040.028541,68.349412,14.919228,79,0 days 00:05:00,108.434606,12.974534,35.577388,162.202573,True


In [8]:
len(combined_cells_landfall_present)

5654

## _track up to the time the cell first hit landfall (after it hits the first landfall, remove all the subsequent cells hitting landfall, we are not interested in it)_

In [9]:
%%time
cells_to_first_landfall = []
for i, cell_idx in enumerate(np.unique(cells_landfall)):
    df = combined_cells_landfall_present[combined_cells_landfall_present['cell'] == cell_idx]
    # # find the index of the first occurrence of True
    idx = df.loc[df['landfall']].index[0]
    # slice the DataFrame to keep only the rows up to and including that index
    df = df.loc[:idx]
    # set all subsequent occurrences of True to False
    df.loc[df['landfall'].shift(-1) == True, 'landfall'] = False
    # print(df)
    cells_to_first_landfall.append(df)
first_landfall = pd.concat(cells_to_first_landfall)

CPU times: user 1.03 s, sys: 13 ms, total: 1.04 s
Wall time: 1.08 s


## _calculate relative time --- relative time = time - time_landfall_

In [10]:
%%time
cells_relative_time = []
for index, cell_id in enumerate(np.unique(first_landfall['cell'])):
    if cell_id:
        # print(cell_id)
        cells = first_landfall[first_landfall['cell'] == cell_id] # get all the cell_ids
    # calculate for the relative time ----- relative time = time - time_landfall
        all_cells_times = cells['timestr']; all_cells_times = pd.to_datetime(all_cells_times.values) # convert timestr to datetime
        landfall_time_each_cell = all_cells_times[::-1][0]
        # relative_time = np.absolute(all_cells_times - landfall_time_each_cell) # relative time
        relative_time = np.subtract(landfall_time_each_cell, all_cells_times)
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time.append(relative_time)

CPU times: user 765 ms, sys: 2.01 ms, total: 767 ms
Wall time: 789 ms


In [11]:
cells_relative_time[:20]

[Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([30, 25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64')]

In [12]:
# cells_relative_time
cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
# first_landfall['relative_time'] = cells_relative_times
# cells_relative_times.values
first_landfall['relative_time'] = cells_relative_times.values

In [13]:
len(first_landfall)

2416

In [14]:
first_landfall.to_csv('../saved-files/threshold-all/all-cells-first_landfall-threshold-all.csv', index=False)

In [ ]:
# final = first_landfall[['cell', 'latitude', 'longitude', 'distance', 'max dBZ','mean dBZ', 'area' ,'timestr', 'landfall']]
# final

# END 